In [1]:
from haystack.telemetry import tutorial_running
tutorial_running(27)

In [2]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

In [3]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]

In [4]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model='sentence-transformers/all-MiniLM-L6-v2')
doc_embedder.warm_up()

In [5]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings['documents'])

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

151

In [6]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
text_embedder = SentenceTransformersTextEmbedder(model='sentence-transformers/all-MiniLM-L6-v2')

In [7]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store)

In [8]:
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage

template = [ChatMessage.from_user("""
Given the following information, answer the question.

Context:
{% for document in documents %}
  {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
""")]

prompt_builder = ChatPromptBuilder(template=template)

In [9]:
import os 
from getpass import getpass
from haystack.components.generators.chat import OpenAIChatGenerator

if "OPENAI_API_KEY" not in os.environ:
  os.environ['OPENAI_API_KEY'] = getpass('Enter OpenAI API key: ')

chat_generator = OpenAIChatGenerator(model='gpt-3.5-turbo')

In [10]:
from haystack import Pipeline

basic_rag_pipeline = Pipeline()

basic_rag_pipeline.add_component('text_embedder', text_embedder)
basic_rag_pipeline.add_component('retriever', retriever)
basic_rag_pipeline.add_component('prompt_builder', prompt_builder)
basic_rag_pipeline.add_component('llm', chat_generator)

In [11]:
basic_rag_pipeline.connect('text_embedder.embedding', 'retriever.query_embedding')
basic_rag_pipeline.connect('retriever', 'prompt_builder')
basic_rag_pipeline.connect('prompt_builder.prompt', 'llm.messages')

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: OpenAIChatGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])

In [12]:
question = "What does Rhodes Statue look like?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0].text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Rhodes statue was of the Greek sun-god Helios and it was described as being approximately 33 meters (108 feet) tall. It had a standard rendering of a head and face with curly hair and evenly spaced spikes of bronze or silver flame radiating, similar to the images found on contemporary Rhodian coins.
